In [1]:
import urllib
import os 
import concurrent.futures

import pandas as pd
from bs4 import BeautifulSoup

# Download assemblies/contigs from HMP2 study

Got a HMTL of the contigs download links from HMP2 from https://ibdmdb.org/downloads/html/products_MGX_2017-08-12.html

Data was processed by the following pipeline https://github.com/biobakery/hmp2_workflows/tree/master/hmp2_workflows

TLDR: 
1. Reads were processed using kneaddata. Trimmomatic and then removal of contaminants with bowtie2 https://huttenhower.sph.harvard.edu/kneaddata/
2. Assembled with MEGAHIT

In [2]:
soup = BeautifulSoup(open("products_MGX_2017-08-12.html"), 'html.parser')
a_results = soup.find_all("a")

In [3]:
download_links = []
download_paths = []
directory_path = "/orcd/data/braakman/001/data/assemblies/hmp2"

for element in a_results:
    download_link = element.get('href')
    if "contigs" in download_link:
        name = os.path.basename(download_link)
        file_path = os.path.join(directory_path, name)
        download_links.append(download_link)
        download_paths.append(file_path)

In [64]:
def download_file(link: str, path: str):
    try:
        urllib.request.urlretrieve(link, path)
        return (True, link)
    except:
        return (False, link)

with concurrent.futures.ThreadPoolExecutor() as executor:
    downloading = executor.map(download_file, download_links, download_paths)
    
    download_df = pd.DataFrame(downloading, columns = ["Downloaded", "Link"])

download_df

,Downloaded,Link
0,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
1,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
2,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
3,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
4,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
...,...,...
1333,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
1334,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
1335,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...
1336,True,https://g-227ca.190ebd.75bc.data.globus.org/ib...


In [70]:
print(len(download_links))
download_df["Downloaded"].sum() == len(download_links)

1338


np.True_

All assemblies/contigs have been downloaded